## Libraries, Dependencies and Parameters


_Install Required Libraries_


In [ ]:
%pip install openprompt
%pip install evaluate
%pip install adapter-transformers==3.1.0

_Dependencies_


In [ ]:
import torch
import time
import evaluate
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
from datasets import load_metric
from openprompt import PromptDataLoader, PromptForClassification
from openprompt.data_utils import InputExample
from openprompt.prompts import MixedTemplate, SoftVerbalizer
from openprompt.plms.utils import TokenizerWrapper
from transformers import AdamW, get_linear_schedule_with_warmup, XLMRobertaConfig, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForMaskedLM, set_seed, AdapterConfig
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE
from prettytable import PrettyTable

In [ ]:
warnings.filterwarnings('ignore')

_Parameters_


In [ ]:
model_type = "XLM-R"
technique = "Sentiment" #@param ["Sentiment", "Humor", "Hate-Speech"]
over_sampling_technique = "ROS"
sampling_strategy = "1:0.25:0.25"
oversample_dataset = False
use_cuda = True
no_of_labels = 4 #Sentiment:4, Humor:2, Hate-Speech:3
validation_size = (1/9)
test_size = 0.1
split_random_state = 42
max_seq_length = 128
batch_size = 32
num_train_epochs = 20
training_seed = 42 #@param [8, 42, 77]
script = "Char-Script-1.0"

## Dataset Preprocessing


In [ ]:
df = pd.read_csv("/kaggle/input/dravidian-english-cmcs-dataset/kannada-english-sentiment-hate-speech.csv")
df = df[['Sentence', technique, script]]
df.columns = ['Sentence', 'Label', script]
df = df[df["Label"] != "Not-Kannada"]

*Label Encoding*

In [ ]:
df['Label'], uniq = pd.factorize(df['Label'])

*Train, Validation & Test Split*

In [ ]:
X, y = df[['Sentence', script]], df[['Label']]
stratifying_col = y["Label"]
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=test_size, stratify=stratifying_col, random_state=split_random_state)
stratifying_col = y_rem["Label"]
X_train, X_validation, y_train, y_validation = train_test_split(X_rem, y_rem, test_size=validation_size, stratify=stratifying_col, random_state=split_random_state)

In [ ]:
print("Train : Rows =", X_train.shape[0], ", Columns = ", X_train.shape[1])
print("y_train : Rows =", y_train.shape[0], ", Columns = ", y_train.shape[1])
print("X_validation : Rows =", X_validation.shape[0], ", Columns = ", X_validation.shape[1])
print("y_validation : Rows =", y_validation.shape[0], ", Columns = ", y_validation.shape[1])
print("X_test : Rows =", X_test.shape[0], ", Columns = ", X_test.shape[1])
print("y_test : Rows =", y_test.shape[0], ", Columns = ", y_test.shape[1])

In [ ]:
print("Labels :", ['Positive', 'Neutral', 'Negative', 'Conflict'])
print("Train :", y_train.groupby('Label').size().tolist())
print("Validation :", y_validation.groupby('Label').size().tolist())
print("Test :", y_test.groupby('Label').size().tolist())

*Oversampling*

In [ ]:
def apply_oversampling(x, y):
  (unique, counts) = np.unique(y, axis=0, return_counts=True)
  print("Class Distribution Without Oversampling", counts)

  # define oversampling strategy
  if (over_sampling_technique == ""):
    return x, y
  elif (over_sampling_technique == "ROS"):
    if (technique=="Humor"):
        oversample = RandomOverSampler(sampling_strategy = float(sampling_strategy))
    elif (technique=="Hate-Speech"):
        sampling_ratio = sampling_strategy.split(":")
        oversample = RandomOverSampler(sampling_strategy = {
          0:int(counts[0]*float(sampling_ratio[0])), 
          1:int(counts[0]*float(sampling_ratio[1])), 
          2:int(counts[0]*float(sampling_ratio[2]))
          })
    elif (technique=="Sentiment"):
        sampling_ratio = sampling_strategy.split(":")
        oversample = RandomOverSampler(sampling_strategy = {
          0:int(counts[1]*float(sampling_ratio[0])), 
          1:int(counts[1]*float(sampling_ratio[1])), 
          2:int(counts[1]*float(sampling_ratio[2])),
          3:int(counts[1]*float(sampling_ratio[3]))
          })
  elif (over_sampling_technique == "ADASYN"):
    oversample = ADASYN(sampling_strategy="minority")
  elif (over_sampling_technique == "SMOTE"):
    oversample = SMOTE()
  elif (over_sampling_technique == "BorderlineSMOTE"):
    oversample = BorderlineSMOTE()

  # fit and apply the transform
  X_over, y_over = oversample.fit_resample(x, y)

  (unique, counts) = np.unique(y_over, axis=0, return_counts=True)
  print("Class Distribution After Oversampling", counts)

  return X_over, y_over

In [ ]:
if oversample_dataset:
  # apply oversampling
  X_train = np.array(X_train).reshape(-1, 1)
  X_train, y_train = apply_oversampling(X_train, y_train)
  X_train = [x[0] for x in X_train.tolist()]

## OpenPrompt


In [ ]:
set_seed(training_seed)
torch.backends.cudnn.deterministic = True 

In [ ]:
class MLMTokenizerWrapper(TokenizerWrapper):
    add_input_keys = ['input_ids', 'attention_mask', 'token_type_ids']

    @property
    def mask_token(self):
        return self.tokenizer.mask_token

    @property
    def mask_token_ids(self):
        return self.tokenizer.mask_token_id

    @property
    def num_special_tokens_to_add(self):
        if not hasattr(self, '_num_specials'):
            self._num_specials = self.tokenizer.num_special_tokens_to_add()
        return self._num_specials

    def tokenize_one_example(self, wrapped_example, teacher_forcing):
        wrapped_example, others = wrapped_example
        encoded_tgt_text = []
        if 'tgt_text' in others:
            tgt_text = others['tgt_text']
            if isinstance(tgt_text, str):
                tgt_text = [tgt_text]
            for t in tgt_text:
                encoded_tgt_text.append(self.tokenizer.encode(t, add_special_tokens=False))

        mask_id = 0 # the i-th the mask token in the template.

        encoder_inputs = defaultdict(list)
        for piece in wrapped_example:
            if piece['loss_ids']==1:
                if teacher_forcing: # fill the mask with the tgt task
                    raise RuntimeError("Masked Language Model can't perform teacher forcing training!")
                else:
                    encode_text = [self.mask_token_ids]
                mask_id += 1

            if piece['text'] in self.special_tokens_maps.keys():
                to_replace = self.special_tokens_maps[piece['text']]
                if to_replace is not None:
                    piece['text'] = to_replace
                else:
                    raise KeyError("This tokenizer doesn't specify {} token.".format(piece['text']))

            if 'soft_token_ids' in piece and piece['soft_token_ids']!=0:
                encode_text = [0] # can be replace by any token, since these token will use their own embeddings
            else:
                encode_text = self.tokenizer.encode(piece['text'], add_special_tokens=False)

            encoding_length = len(encode_text)
            encoder_inputs['input_ids'].append(encode_text)
            for key in piece:
                if key not in ['text']:
                    encoder_inputs[key].append([piece[key]]*encoding_length)

        encoder_inputs = self.truncate(encoder_inputs=encoder_inputs)
        # delete shortenable ids
        encoder_inputs.pop("shortenable_ids")
        encoder_inputs = self.concate_parts(input_dict=encoder_inputs)
        encoder_inputs = self.add_special_tokens(encoder_inputs=encoder_inputs)
        # create special input ids
        encoder_inputs['attention_mask'] = [1] *len(encoder_inputs['input_ids'])
        if self.create_token_type_ids:
            encoder_inputs['token_type_ids'] = [0] *len(encoder_inputs['input_ids'])
        # padding
        encoder_inputs = self.padding(input_dict=encoder_inputs, max_len=self.max_seq_length, pad_id_for_inputs=self.tokenizer.pad_token_id)

        if len(encoded_tgt_text) > 0:
            encoder_inputs = {**encoder_inputs, "encoded_tgt_text": encoded_tgt_text}# convert defaultdict to dict
        else:
            encoder_inputs = {**encoder_inputs}
        return encoder_inputs

In [ ]:
def load_plm(model_name, model_path):   
    model_config = XLMRobertaConfig.from_pretrained(model_path)
    model = XLMRobertaForMaskedLM.from_pretrained(model_path, config=model_config)
    tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)
    wrapper = MLMTokenizerWrapper
    return model, tokenizer, wrapper

_Load Pre-trained Language Model (PLM)_


In [ ]:
plm, tokenizer, wrapper_class = load_plm("xlm", "xlm-roberta-base")

**Adapter Configuration**

In [ ]:
adapter_name = "All_Sentiment_Kannada"
adapter_config = AdapterConfig.load("houlsby") #@param = ["pfeiffer", "houlsby"]
# adapter_config.leave_out.extend([9, 10, 11])
plm.add_adapter(adapter_name, config=adapter_config)
plm.set_active_adapters(adapter_name)
plm.train_adapter(adapter_name)

**Prompt Template, Verbalizer and PLM**


*Prompt Model for Latin Script*

In [ ]:
template_latin = '{"placeholder": "text_a"}. {"soft": "The"} {"soft": "sentiment"} {"soft": "or"} {"soft": "the"} {"soft": "feeling"} {"soft": "of"} {"soft": "the"} {"soft": "given"} {"soft": "sentence"} {"soft": "can"} {"soft": "be"} {"soft": "classified"} {"soft": "as"} {"soft": "positive"} {"soft": ","} {"soft": "negative"} {"soft": "or"} {"soft": "neutral"} {"soft": "."} {"soft": "The"} {"soft": "classified"} {"soft": "sentiment"} {"soft": "of"} {"soft": "the"} {"soft": "sentence"} {"soft": "is"} {"mask"}.'
promptTemplate_latin = MixedTemplate(model=plm, text = template_latin, tokenizer = tokenizer)
promptVerbalizer_latin = SoftVerbalizer(tokenizer, plm, num_classes=no_of_labels)
promptModel_latin = PromptForClassification(template = promptTemplate_latin, plm = plm, verbalizer = promptVerbalizer_latin)

*Prompt Model for Sinhala Script*

In [ ]:
template_kannada = '{"placeholder": "text_a"}. {"soft": "The"} {"soft": "sentiment"} {"soft": "or"} {"soft": "the"} {"soft": "feeling"} {"soft": "of"} {"soft": "the"} {"soft": "given"} {"soft": "sentence"} {"soft": "can"} {"soft": "be"} {"soft": "classified"} {"soft": "as"} {"soft": "positive"} {"soft": ","} {"soft": "negative"} {"soft": "or"} {"soft": "neutral"} {"soft": "."} {"soft": "The"} {"soft": "classified"} {"soft": "sentiment"} {"soft": "of"} {"soft": "the"} {"soft": "sentence"} {"soft": "is"} {"mask"}.'
promptTemplate_kannada = MixedTemplate(model=plm, text = template_kannada, tokenizer = tokenizer)
promptVerbalizer_kannada = SoftVerbalizer(tokenizer, plm, num_classes=no_of_labels)
promptModel_kannada = PromptForClassification(template = promptTemplate_kannada, plm = plm, verbalizer = promptVerbalizer_kannada)

*Prompt Model for Mixed Script*

In [ ]:
template_mixed = '{"placeholder": "text_a"}. {"soft": "The"} {"soft": "sentiment"} {"soft": "or"} {"soft": "the"} {"soft": "feeling"} {"soft": "of"} {"soft": "the"} {"soft": "given"} {"soft": "sentence"} {"soft": "can"} {"soft": "be"} {"soft": "classified"} {"soft": "as"} {"soft": "positive"} {"soft": ","} {"soft": "negative"} {"soft": "or"} {"soft": "neutral"} {"soft": "."} {"soft": "The"} {"soft": "classified"} {"soft": "sentiment"} {"soft": "of"} {"soft": "the"} {"soft": "sentence"} {"soft": "is"} {"mask"}.'
promptTemplate_mixed = MixedTemplate(model=plm, text = template_mixed, tokenizer = tokenizer)
promptVerbalizer_mixed = SoftVerbalizer(tokenizer, plm, num_classes=no_of_labels)
promptModel_mixed = PromptForClassification(template = promptTemplate_mixed, plm = plm, verbalizer = promptVerbalizer_mixed)

## Create DataLoaders


In [ ]:
X_train, y_train = X_train.values.tolist(), y_train.values.tolist()
X_validation, y_validation = X_validation.values.tolist(), y_validation.values.tolist()
X_test, y_test = X_test.values.tolist(), y_test.values.tolist()

_Train Dataloader_


In [ ]:
train_examples_latin=[]
train_examples_kannada=[]
train_examples_mixed=[]

for i in range(len(X_train)):
    if(X_train[i][1]=="Latin"):
        train_examples_latin.append(InputExample(guid = i, text_a = X_train[i][0], label = y_train[i][0]))
    elif(X_train[i][1]=="Kannada"):
        train_examples_kannada.append(InputExample(guid = i, text_a = X_train[i][0], label = y_train[i][0]))    
    elif(X_train[i][1]=="Mixed"):
        train_examples_mixed.append(InputExample(guid = i, text_a = X_train[i][0], label = y_train[i][0]))

train_data_loader_latin = PromptDataLoader(
    dataset = train_examples_latin,
    tokenizer = tokenizer,
    template = promptTemplate_latin,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length"
).dataloader
train_data_loader_kannada = PromptDataLoader(
    dataset = train_examples_kannada,
    tokenizer = tokenizer,
    template = promptTemplate_kannada,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length"
).dataloader
train_data_loader_mixed = PromptDataLoader(
    dataset = train_examples_mixed,
    tokenizer = tokenizer,
    template = promptTemplate_mixed,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length",
).dataloader

_Validation Dataloader_


In [ ]:
validation_examples_latin=[]
validation_examples_kannada=[]
validation_examples_mixed=[]

for i in range(len(X_validation)):
    if(X_validation[i][1]=="Latin"):
        validation_examples_latin.append(InputExample(guid = i, text_a = X_validation[i][0], label = y_validation[i][0]))
    elif(X_validation[i][1]=="Kannada"):
        validation_examples_kannada.append(InputExample(guid = i, text_a = X_validation[i][0], label = y_validation[i][0]))    
    elif(X_validation[i][1]=="Mixed"):
        validation_examples_mixed.append(InputExample(guid = i, text_a = X_validation[i][0], label = y_validation[i][0]))

validation_data_loader_latin = PromptDataLoader(
    dataset = validation_examples_latin,
    tokenizer = tokenizer,
    template = promptTemplate_latin,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length"
).dataloader
validation_data_loader_kannada = PromptDataLoader(
    dataset = validation_examples_kannada,
    tokenizer = tokenizer,
    template = promptTemplate_kannada,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length"
).dataloader
validation_data_loader_mixed = PromptDataLoader(
    dataset = validation_examples_mixed,
    tokenizer = tokenizer,
    template = promptTemplate_mixed,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length",
).dataloader

_Test Dataloader_


In [ ]:
test_examples_latin=[]
test_examples_kannada=[]
test_examples_mixed=[]

for i in range(len(X_test)):
    if(X_test[i][1]=="Latin"):
        test_examples_latin.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))
    elif(X_test[i][1]=="Kannada"):
        test_examples_kannada.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))    
    elif(X_test[i][1]=="Mixed"):
        test_examples_mixed.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))

test_data_loader_latin = PromptDataLoader(
    dataset = test_examples_latin,
    tokenizer = tokenizer,
    template = promptTemplate_latin,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length"
).dataloader
test_data_loader_kannada = PromptDataLoader(
    dataset = test_examples_kannada,
    tokenizer = tokenizer,
    template = promptTemplate_kannada,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length"
).dataloader
test_data_loader_mixed = PromptDataLoader(
    dataset = test_examples_mixed,
    tokenizer = tokenizer,
    template = promptTemplate_mixed,
    tokenizer_wrapper_class=wrapper_class,
    batch_size=batch_size,
    max_seq_length=max_seq_length,
    truncation=True,
    padding="max_length",
).dataloader

## Create Optimizer and Scheduler

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
betas = (0.9, 0.999)
eps = 1e-08
num_training_steps = len(X_train) * num_train_epochs # After the num_training_steps lr will be 0 

# ==== Latin ====
latin_num_warmup_steps = 250
latin_sp_lr = 1e-3
latin_sv_gp_1_lr = 4e-5
latin_sv_gp_2_lr = 4e-4

# ==== Sinhala ====
sinhala_num_warmup_steps = 1500
sinhala_sp_lr = 1e-2
sinhala_sv_gp_1_lr = 2e-2
sinhala_sv_gp_2_lr = 2e-2

# ==== Mixed ====
mixed_num_warmup_steps = 1250
mixed_sp_lr = 1e-2
mixed_sv_gp_1_lr = 2e-2
mixed_sv_gp_2_lr = 2e-2

# ==== Shared Adapter ====
adapter_wd_1 = 1e-2
adapter_wd_2 = 0
adapter_num_warmup_steps = 500
adapter_lr = 5e-4

In [ ]:
def create_optimizer_and_scheduler(promptModel, sp_lr, sv_gp_1_lr, sv_gp_2_lr, num_warmup_steps, num_training_steps):   
    sp_optimizer_grouped_parameters = [{'params': [p for n,p in promptModel.template.named_parameters() if "raw_embedding" not in n]}]
    sp_optimizer = AdamW(sp_optimizer_grouped_parameters, lr=sp_lr, betas=betas, eps=eps)
    sp_scheduler = get_linear_schedule_with_warmup(sp_optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) # After the num_training_steps lr will be 0

    sv_optimizer_grouped_parameters = [{'params': promptModel.verbalizer.group_parameters_1, "lr":sv_gp_1_lr}, {'params': promptModel.verbalizer.group_parameters_2, "lr":sv_gp_2_lr}]
    sv_optimizer = AdamW(sv_optimizer_grouped_parameters,betas=betas,eps=eps)
    sv_scheduler = get_linear_schedule_with_warmup(sv_optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
    
    adapter_optimizer_grouped_parameters = [{'params': [p for n, p in promptModel.plm.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': adapter_wd_1}, {'params': [p for n, p in promptModel.plm.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': adapter_wd_2}]
    adapter_optimizer = AdamW(adapter_optimizer_grouped_parameters, lr=adapter_lr, betas=betas, eps=eps)
    adapter_scheduler = get_linear_schedule_with_warmup(adapter_optimizer, num_warmup_steps=adapter_num_warmup_steps, num_training_steps=num_training_steps)
    
    return sp_optimizer, sp_scheduler, sv_optimizer, sv_scheduler, adapter_optimizer, adapter_scheduler

## Define Evaluation Methods


In [ ]:
def predict(promptModel, dataloader):
    if use_cuda:
        promptModel=promptModel.cuda()
        
    promptModel.eval()
    allpreds = []
    alllabels = []

    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    return allpreds,alllabels

In [ ]:
def compute_metrics(allpreds,alllabels):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    predictions, labels = allpreds,alllabels
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_precision = metric1.compute(predictions=predictions, references=labels, average="macro")["precision"]
    macro_recall = metric2.compute(predictions=predictions, references=labels, average="macro")["recall"]
    macro_f1 = metric3.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1, "macro_precision": macro_precision, "macro_recall": macro_recall, "macro_f1": macro_f1}

In [ ]:
def evaluate(promptModel, dataloader):
    allpreds,alllabels=predict(promptModel, dataloader)
    return compute_metrics(allpreds,alllabels)

In [ ]:
def calculate_loss_and_f1(promptModel, dataloader):
    if use_cuda:
        promptModel=promptModel.cuda()
    
    promptModel.eval()
    allpreds = []
    alllabels = []
    total_loss = 0

    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        loss = (torch.nn.CrossEntropyLoss())(logits, labels)
        total_loss += loss.item()
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
        alllabels.extend(labels.cpu().tolist())
    
    macro_f1 = load_metric("f1").compute(predictions=allpreds, references=alllabels, average="macro")["f1"]
    return macro_f1, (total_loss/len(dataloader))

## Train


_Training Parameters_

In [ ]:
loss_func = torch.nn.CrossEntropyLoss()
pbar_update_freq = 10

max_grad_norm = 1.0
gradient_accumulation_steps = 1

val_metric = "macro_f1"
early_stop_epoch_thresh = num_train_epochs

In [ ]:
def train(script, promptModel, sp_lr, sv_gp_1_lr, sv_gp_2_lr, num_warmup_steps, train_data_loader, validation_data_loader):
    if use_cuda:
        promptModel=promptModel.cuda()
    set_seed(training_seed)
    
    # Train Parameters    
    tot_loss = 0
    log_loss = 0
    best_val_acc = 0

    glb_step = 0
    actual_step = 0
    leave_training = False
    best_epoch = -1
    
    epoch_traces = []
    acc_traces = []
    validation_loss_traces = []
    
    num_training_steps = len(train_data_loader) * num_train_epochs # After the num_training_steps lr will be 0 
    
    # Optimizer and Scheduler
    sp_optimizer, sp_scheduler, sv_optimizer, sv_scheduler, adapter_optimizer, adapter_scheduler = create_optimizer_and_scheduler(promptModel, sp_lr, sv_gp_1_lr, sv_gp_2_lr, num_warmup_steps, num_training_steps)
    
    # Train
    pbar = tqdm(total=num_training_steps, desc="Train")
    for epoch in range(num_train_epochs):
        print(f"Begin Epoch {epoch}")
        epoch_start_time = time.time()
        for step, inputs in enumerate(train_data_loader):
            if use_cuda:
                inputs = inputs.cuda()
            logits = promptModel(inputs)
            labels = inputs['label']
            loss = loss_func(logits, labels)
            loss = loss / gradient_accumulation_steps
            loss.backward()
            tot_loss += loss.item()
            actual_step += 1

            if actual_step % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(promptModel.parameters(), max_grad_norm)
                glb_step += 1

                if glb_step % pbar_update_freq == 0:              
                    aveloss = (tot_loss - log_loss)/pbar_update_freq
                    pbar.update(pbar_update_freq)
                    pbar.set_postfix({'Average Loss': aveloss, "Epoch": epoch})
                    log_loss = tot_loss

                if sp_optimizer is not None:
                    sp_optimizer.step()
                    sp_optimizer.zero_grad()
                if sp_scheduler is not None:
                    sp_scheduler.step()
                if sv_optimizer is not None:
                    sv_optimizer.step()
                    sv_optimizer.zero_grad()
                if sv_scheduler is not None:
                    sv_scheduler.step()
                if adapter_optimizer is not None:
                    adapter_optimizer.step()
                    adapter_optimizer.zero_grad()
                if adapter_scheduler is not None:
                    adapter_scheduler.step()

            if glb_step > num_training_steps:
                leave_training = True
                break

        val_acc, val_loss = calculate_loss_and_f1(promptModel, validation_data_loader)
        epoch_traces.append(epoch)
        acc_traces.append(val_acc)
        validation_loss_traces.append(val_loss)
        print("Validation: [Epoch: {}, Macro F1: {}, Validation Loss: {}, Time per Epoch: {}]".format(epoch, val_acc, val_loss, time.time()-epoch_start_time), flush=True)

        if val_acc > best_val_acc:
            torch.save(promptModel.state_dict(),f"best_model_{script}.ckpt")
            best_val_acc = val_acc
            best_epoch = epoch

        elif (epoch - best_epoch) >= early_stop_epoch_thresh:
            print("Training stopped early at Epoch: %d" % epoch)
            break  # Terminate the training loop

        if leave_training:
            break
    
    promptModel.load_state_dict(torch.load(f"best_model_{script}.ckpt"))
    if use_cuda:
        promptModel = promptModel.cuda()
        
    return promptModel, epoch_traces, acc_traces, validation_loss_traces

**Train the Models**


*Latin*

In [ ]:
promptModel_latin, epoch_traces_latin, acc_traces_latin, validation_loss_traces_latin = train(
    script = "latin", promptModel = promptModel_latin, 
    sp_lr = latin_sp_lr, sv_gp_1_lr = latin_sv_gp_1_lr, sv_gp_2_lr = latin_sv_gp_2_lr, num_warmup_steps = latin_num_warmup_steps,
    train_data_loader = train_data_loader_latin, validation_data_loader = validation_data_loader_latin
)

*Sinhala*

In [ ]:
promptModel_kannada, epoch_traces_kannada, acc_traces_kannada, validation_loss_traces_kannada = train(
    script = "sinhala", promptModel = promptModel_kannada, 
    sp_lr = sinhala_sp_lr, sv_gp_1_lr = sinhala_sv_gp_1_lr, sv_gp_2_lr = sinhala_sv_gp_2_lr, num_warmup_steps = sinhala_num_warmup_steps,
    train_data_loader = train_data_loader_kannada, validation_data_loader = validation_data_loader_kannada
)

*Mixed*

In [ ]:
promptModel_mixed, epoch_traces_mixed, acc_traces_mixed, validation_loss_traces_mixed = train(
    script = "mixed", promptModel = promptModel_mixed, 
    sp_lr = mixed_sp_lr, sv_gp_1_lr = mixed_sv_gp_1_lr, sv_gp_2_lr = mixed_sv_gp_2_lr, num_warmup_steps = mixed_num_warmup_steps,
    train_data_loader = train_data_loader_mixed, validation_data_loader = validation_data_loader_mixed
)

## Validation


**Script-Wise Validation**

In [ ]:
t = PrettyTable()
t.field_names = ['Script', 'Accuracy', 'Precision', 'Recall', 'F1', 'Macro Precision', 'Macro Recall', 'Macro F1']

*Latin*

In [ ]:
val_preds_latin, val_labels_latin = predict(promptModel_latin, validation_data_loader_latin)
res = compute_metrics(val_preds_latin, val_labels_latin)
t.add_row(["Latin", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])

 *Sinhala*

In [ ]:
val_preds_kannada, val_labels_kannada = predict(promptModel_kannada, validation_data_loader_kannada)
res = compute_metrics(val_preds_kannada, val_labels_kannada)
t.add_row(["Kannada", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])

*Mixed*

In [ ]:
val_preds_mixed, val_labels_mixed = predict(promptModel_mixed, validation_data_loader_mixed)
res = compute_metrics(val_preds_mixed, val_labels_mixed)
t.add_row(["Mixed", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']], divider=True)

**Overall Validation**

In [ ]:
val_preds, val_labels = (val_preds_latin+val_preds_kannada+val_preds_mixed), (val_labels_latin+val_labels_kannada+val_labels_mixed)
res = compute_metrics(val_preds, val_labels)
t.add_row(["Overall", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])

*Plot Validation Graphs*

In [ ]:
def plot_validation_graphs(script, epoch_traces, acc_traces, validation_loss_traces):
    plt.figure(figsize=(20,5))
    
    plt.subplot(1, 2, 1) # row 1, col 2 index 1
    plt.plot(range(len(epoch_traces)), acc_traces)
    plt.xlabel('Epoch')
    plt.ylabel('Macro F1-Score')
    plt.title(script + ' : Epoch vs Validation Macro F1-Score')
    plt.xticks(range(len(epoch_traces)), epoch_traces)
    
    plt.subplot(1, 2, 2) # index 2
    plt.plot(range(len(epoch_traces)), validation_loss_traces)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(script + ' : Epoch vs Validation Loss')
    plt.xticks(range(len(epoch_traces)), epoch_traces)
    
    plt.show()

In [ ]:
print(t.get_string(fields=["Script", "Accuracy", "Macro F1"]))
plot_validation_graphs("Latin", epoch_traces_latin, acc_traces_latin, validation_loss_traces_latin)
plot_validation_graphs("Kannada", epoch_traces_kannada, acc_traces_kannada, validation_loss_traces_kannada)
plot_validation_graphs("Mixed", epoch_traces_mixed, acc_traces_mixed, validation_loss_traces_mixed)

## Test


In [ ]:
t = PrettyTable()
t.field_names = ['Script', 'Accuracy', 'Precision', 'Recall', 'F1', 'Ma. Precision', 'Ma. Recall', 'Ma. F1']

**Script-Wise Evaluation**


_Latin Script_


In [ ]:
test_preds_latin, test_labels_latin = predict(promptModel_latin, test_data_loader_latin)
res = compute_metrics(test_preds_latin, test_labels_latin)
res = {key : round(res[key], 4) for key in res}
t.add_row(["Latin", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])

_Sinhala Script_


In [ ]:
test_preds_kannada, test_labels_kannada = predict(promptModel_kannada, test_data_loader_kannada)
res = compute_metrics(test_preds_kannada, test_labels_kannada)
res = {key : round(res[key], 4) for key in res}
t.add_row(["Kannada", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])

_Mixed Script_


In [ ]:
test_preds_mixed, test_labels_mixed = predict(promptModel_mixed, test_data_loader_mixed)
res = compute_metrics(test_preds_mixed, test_labels_mixed)
res = {key : round(res[key], 4) for key in res}
t.add_row(["Mixed", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']], divider=True)

**Overall Evaluation**

In [ ]:
test_preds, test_labels = (test_preds_latin+test_preds_kannada+test_preds_mixed), (test_labels_latin+test_labels_kannada+test_labels_mixed)
res = compute_metrics(test_preds, test_labels)
res = {key : round(res[key], 4) for key in res}
t.add_row(["Overall", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])

In [ ]:
print(t)